In [1]:
# imports

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI
import ollama
import anthropic
import gradio as gr

In [2]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'
MODEL_ANTHROPIC = 'claude-3-7-sonnet-latest'

In [3]:
# set up environment

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')


openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")


# Connect to OpenAI, Anthropic and ollama

openai = OpenAI()

claude = anthropic.Anthropic()

ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
API key looks good so far


In [4]:
system_prompt = """
You are helping me in an important data science role where I have to build models based on meta-data without seeing the true data. To test that my models do the correct thing, I
need you to generate datasets  when given a description of what they look like. You will respond in either with either a JSON or a CSV depending on what is being asked. If 
it is unclear what should be the range or type of data in a given column, you should prompt the user to give more information. 
"""

In [5]:

def ask_for_dataset():
    return input("Do you need a dataset?: ")

In [70]:
def stream_claude(prompt, history):

    
    result = claude.messages.stream(
        model="claude-3-haiku-20240307",
        max_tokens=1000,
        temperature=0.7,
        system=system_prompt,
        messages=[
            history + {"role": "user", "content": prompt},
        ],
    )
    response = ""
    with result as stream:
        for text in stream.text_stream:
            response += text or ""
            yield response


       

In [73]:

def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]

    print("History is:")
    print(history)
    print("And messages is:")
    print(messages)

    stream = openai.chat.completions.create(model=MODEL_GPT, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [74]:
gr.ChatInterface(
    fn=chat,
    type="messages",  # 🚨 critical: this means `fn` receives a list of messages
    title="Chat with Claude",
    description="Talk to Claude 3 Haiku using the Anthropics API",
).launch()

* Running on local URL:  http://127.0.0.1:7887
* To create a public link, set `share=True` in `launch()`.


History is:
[]
And messages is:
[{'role': 'system', 'content': '\nYou are helping me in an important data science role where I have to build models based on meta-data without seeing the true data. To test that my models do the correct thing, I\nneed you to generate datasets  when given a description of what they look like. You will respond in either with either a JSON or a CSV depending on what is being asked. If \nit is unclear what should be the range or type of data in a given column, you should prompt the user to give more information. \n'}, {'role': 'user', 'content': 'hey'}]
History is:
[{'role': 'user', 'metadata': None, 'content': 'hey', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': "Hello! How can I assist you today? If you need help generating a dataset, please provide a description of what you're looking for.", 'options': None}]
And messages is:
[{'role': 'system', 'content': '\nYou are helping me in an important data science role where I have to build